In [1]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

MODEL_NAME = "llama3.2:3b"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
RAG_TEMPLATE = """
You are an assistant for question-answering tasks about companies. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep the answer informative but concise.
Question: {question} 
Context: {context} 
Answer:
"""

Libraries and configurations loaded successfully.


In [10]:
embeddings = OllamaEmbeddings(model=MODEL_NAME)
vector_store = InMemoryVectorStore(embeddings)
model = OllamaLLM(model=MODEL_NAME)


In [6]:
def load_page(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        loader = SeleniumURLLoader(urls=[url])
        documents = loader.load()
        return documents
    except Exception as e:
        print(f"Error loading page: {str(e)}")
        return []

url = "echelonconsulting.com"
documents = load_page(url)

print(f"Scraped {len(documents)} document(s) from {url}")
if documents:
    print("Sample content (first 200 characters):")
    print(documents[0].page_content[:200])
else:
    print("No content scraped.")


Scraped 1 document(s) from echelonconsulting.com
Sample content (first 200 characters):
Forging Long-Term Partnerships

Leaner And More Agile High-Performing Teams

Aligning Your Business Needs With Modern Technology

Helping You Reach Peak Performance

Echelon Consulting is...



Strategic

A full-service, full lifecycle application development and application strategy house.



Understanding

A firm that understands people, process and technology.



Ethical

An ethical firm that forges long-term partnerships with our clients.



Seasoned

A seasoned team of full-time software engineers based in Chicago, with competencies that span a wide range of technologies and platforms, including mobile.



Accessible

A professional support team that offers 24x7x365 support.



People Focused

A firm that puts people at the forefront of our business.



Custom application development and strategy consulting since 1997...



Echelon origins: Learn about us...

Learn About Us



Working with our 

In [7]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        add_start_index=True
    )
    return text_splitter.split_documents(documents)

chunked_documents = split_text(documents)

print(f"Split into {len(chunked_documents)} chunks")
if chunked_documents:
    print("Sample chunk (first 200 characters):")
    print(chunked_documents[0].page_content[:200])
    print(f"Metadata: {chunked_documents[0].metadata}")


Split into 4 chunks
Sample chunk (first 200 characters):
Forging Long-Term Partnerships

Leaner And More Agile High-Performing Teams

Aligning Your Business Needs With Modern Technology

Helping You Reach Peak Performance

Echelon Consulting is...



Strate
Metadata: {'source': 'https://echelonconsulting.com', 'title': 'Home | Echelon Consulting, L.L.C.', 'description': 'Embracing the unknown. Developing software solutions and strategies for multi-national clients since 1997.', 'language': 'en', 'start_index': 0}


In [8]:
def index_docs(documents):
    vector_store.add_documents(documents)
    print(f"Indexed {len(documents)} documents into vector store.")

index_docs(chunked_documents)


Indexed 4 documents into vector store.


In [9]:
def retrieve_docs(query):
    return vector_store.similarity_search(query)

def answer_question(question, context):
    prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
    chain = prompt | model
    return chain.invoke({"question": question, "context": context})

question = "What services does Echelon Consulting offer?"
retrieved_docs = retrieve_docs(question)

print(f"Retrieved {len(retrieved_docs)} documents for query: {question}")
if retrieved_docs:
    print("Sample retrieved content (first 200 characters):")
    print(retrieved_docs[0].page_content[:200])

context = "\n\n".join([doc.page_content for doc in retrieved_docs])
answer = answer_question(question, context) if context else "No information available."

print("\nAnswer:")
print(answer)


Retrieved 4 documents for query: What services does Echelon Consulting offer?
Sample retrieved content (first 200 characters):
Echelon Consulting, L.L.C. 10 S. LaSalle St., Suite 2320, Chicago, IL 60603

+1 312-642-2405

inquiries@echelon.org

Copyright © 2025 Echelon Consulting, L.L.C.

Privacy Policy

Cookie Policy

The Ech

Answer:
Based on the retrieved context, I found that Echelon Consulting offers a range of services including:

* Custom application development
* Application strategy consulting
* Leaner and more agile high-performing teams
* Aligning business needs with modern technology
* Helping clients reach peak performance
* Strategic full-service, full lifecycle application development

They also emphasize putting people at the forefront of their business, focusing on transparency, teamwork, and global responsibility.
